# 準備

## GPUを使えるようにする

1. メニューから `Runtime>Change runtime type` を選択する
2. Hardware acceleratorでGPUを選択する

In [ ]:
!nvidia-smi

## Google driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Google Drive上でのディレクトリ作成

コマンドは `% + コマンド` で実現していきます．

Google Driveのルートディレクトリである　`/content/drive/MyDrive`  に移動します．

In [ ]:
%cd /content/drive/MyDrive/

```
MyDrive
|--TDSW_DLWS
    |--ImageClassifier
        |--models
```
という構造でディレクトリを作成します．

In [ ]:
%mkdir TDSW_DLWS TDSW_DLWS/ImageClassifier/ TDSW_DLWS/ImageClassifier/models

ディレクトリを移動します．

In [5]:
%cd TDSW_DLWS/ImageClassifier/models

/content/drive/MyDrive/TDSW_DLWS/ImageClassifier/models


# 学習

## ライブラリのインポート
必要なライブラリ，モジュールを読み込みます．  
それぞれの機能については実際に使用する段階で説明します．


In [12]:
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Flatten, Conv2D, Dense, BatchNormalization, LeakyReLU, Dropout, Activation, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import TensorBoard

## データセットの読み込み＋前処理

### CIFAR-10（画像とラベル）の読み込み
CIFAR-10データセットを読み込みます．
これは下の図のような60,000枚の32x32pxのRGB画像とそのラベル（どのクラスに属するか）のセットです．

</br>

<img src="https://drive.google.com/uc?id=1iHrBQSOBeOHNC9bMZc12IXhdtZk-ihM6" width = 50%>

</br>

CIFAR-10などはKerasに同封されているためサクッと読み込めます．

- x_train: 入力する訓練データ
- x_test: テスト用データ
- y_train: x_trainの各観測が属するクラス0 -- 9
- y_test: x_testの各観測が属するクラス0 -- 9
- cifar10.load_data(): データを読み込む関数

データセットは訓練用とテスト用に分ける必要があります．

In [ ]:
NUM_CLASSES = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print(x_train.shape)

In [ ]:
print(y_train.shape)

### 0 - 255を0.0 - 1.0の32bit floatに変換
ニューラルネットワークの入力は-1.0 -- 1.0で最も機能するため，割り算します．

In [9]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

### one-hotエンコーディング
ラベルデータは，

- 'airplane': 0
- 'automobile': 1
- 'bird': 2
- 'cat': 3

のようになっていますが，**クラス名と数字の大小は本来関係がありません**．  
この大小関係を持ち込まないために，正解が1でそれ以外ゼロのベクトルに変更します．  
ある画像のラベルが2番目（値は1）の場合:  
`[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]`

10クラスあるので，ベクトルの形状は`[50000, 1]`から`[50000, 10]`に変化します．  

</br>

関数はto_categorical(データ, クラス数)を使用します．

In [10]:
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

### データの水増し(今回は割愛)
モデルの精度を高めるためにデータセットの画像に反転，回転，拡大縮小などの処理を加えてデータセットの数を増やします．

In [88]:
# データの水増し
gen = ImageDataGenerator(rotation_range = 20, horizontal_flip = True, height_shift_range = 0.2, width_shift_range = 0.2,zoom_range = 0.2, channel_shift_range = 0.2)
train_gen = gen.flow(x_train, y_train, batch_size=32)

## アーキテクチャのセットアップ
Sequentialモデルではなく，より自由度の高いFunctional APIにて設計していきます．  
あまり記述量も変わらないので，最初からFunctional APIで実装していくことをおすすめします．

### 入力層
入力層には任意の枚数の画像を渡します．  
shapeは入力の次元数を示しています．  
バッチサイズは任意の枚数であるため，指定する必要はありません．

In [64]:
input_layer = Input(shape = (32, 32, 3))

### 畳み込み層
convolution layerを宣言  
kernel_size: カーネル（フィルタ）の縦横幅  
strides: 画像をスキャンするときのステップサイズ  
padding: sameの場合，畳み込み時にできる元画像の空白をゼロで埋める  

In [ ]:
x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(input_layer)
# x = BatchNormalization()(x)
x = LeakyReLU()(x)

# x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same')(x)
# x = BatchNormalization()(x)
# x = LeakyReLU()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
# x = BatchNormalization()(x)
x = LeakyReLU()(x)

# x = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(x)
# x = BatchNormalization()(x)
# x = LeakyReLU()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# 次のDense層の入力が１次元配列のためFlatten層でベクトル化
# 3072 = 32 * 32 * 3
x = Flatten()(x)

# 前の層と密結合したノードを無限に持てるDense層
# 活性化関数LeakyReLU．
# Dropoutの引数はどれだけの割合ゼロするか
# x = Dense(128)(x)
# x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate=0.50)(x)

x = Dense(NUM_CLASSES)(x)

# 出力層を活性化関数softmax（総和１，他クラス分類問題に良い）
output_layer = Activation('softmax')(x)

# モデルとして宣言
model = Model(input_layer, output_layer)
model.summary()

## モデルのコンパイル
普段聞くコンパイルは，  

**プログラム言語->コンピュータが実行可能な形式への変換**  

を指しますが，Kerasにおけるコンパイルは

**訓練プロセスを作る**

ことを指します．

In [66]:
# optimizerをAdamで学習率0.0005で宣言（学習率以外のパラメタ変更不要）
opt = Adam(learning_rate=0.0005)
# 損失関数を多クラス交差エントロピー（多クラス分類問題に良い），評価関数をaccuracy（とすると損失関数のテンソルから自動でcotagoricalに），optimizerを入れてコンパイル
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

## Tensorboardの起動

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## モデルの訓練

In [68]:
tf_callback = TensorBoard(log_dir="logs", histogram_freq=1)

In [ ]:
# batch_size: 各訓練ステップでひとまとまりとして渡す観測数
# epochs: ネットワークに全トレーニングデータを渡す回数
# shuffle: 各訓練ステップで訓練データからランダムに重複なく取り出す
model.fit(x_train, y_train, batch_size=32, epochs=20, shuffle=True, validation_data = (x_test, y_test), callbacks=[tf_callback])
# modelを保存する
model.save('ImageClassifier.h5')

## モデルの検証

In [ ]:
#modelへ保存データを読み込み
model = load_model('ImageClassifier.h5')

######### モデルを評価する
# テストセットに対する評価を行なう
model.evaluate(x_test, y_test)

# テストセットの予測を表示する
CLASSES = np.array(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])
# [10000, 10]それぞれの観測に対する10クラスの確率のベクトル
preds = model.predict(x_test)
# argmax関数で最後の次元の最大値のインデックスがCLASSES配列のインデックスとなる．predsは[10000, 1]
preds_single = CLASSES[np.argmax(preds, axis = -1)]
actual_single = CLASSES[np.argmax(y_test, axis = -1)]

n_to_show = 10
indices = np.random.choice(range(len(x_test)), n_to_show)

fig = plt.figure(figsize=(15, 3))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

counter = 0
correct = 0
for i, idx in enumerate(indices):
    img = x_test[idx]
    ax = fig.add_subplot(1, n_to_show, i + 1)
    ax.axis('off')
    ax.text(0.5, -0.35, 'pred = ' + str(preds_single[idx]), fontsize = 10, ha = 'center', transform = ax.transAxes)
    ax.text(0.5, -0.7, 'act = ' + str(actual_single[idx]), fontsize=10, ha='center', transform=ax.transAxes)
    ax.imshow(img)
    print(str(i) + ': pred = ' + str(preds_single[idx]))
    print(str(i) + ': act = ' + str(actual_single[idx]))
    if preds_single[idx] == actual_single[idx]:
        correct += 1
    counter += 1
print('correct rate is: ' + str(100 * correct / counter))

# ONNXへ変換

In [ ]:
!pip install onnx
!pip install tf2onnx

In [ ]:
!git clone https://github.com/onnx/tensorflow-onnx

In [ ]:
import tensorflow as tf

model = load_model('ImageClassifier.h5')
tf.saved_model.save(model, 'tmp_model')

In [ ]:
!python -m tensorflow-onnx.tf2onnx.convert --saved-model tmp_model --output "ImageClassifier.onnx"